In [ ]:
from glob import glob
import pandas as pd
problemCsvList = glob("../Project_CodeNet/metadata/*.csv")
solutionFolderList = glob("../Project_CodeNet/data/*")


In [ ]:
print(len(problemCsvList))
print(len(solutionFolderList))


In [ ]:
set([val.split("/")[-1].replace(".csv", "") for val in problemCsvList]) ^ set([val.split("/")[-1] for val in solutionFolderList])

In [ ]:
"../Project_CodeNet/metadata/problem_list.csv" in problemCsvList

In [ ]:
problemListCsvDF =pd.read_csv("../Project_CodeNet/metadata/problem_list.csv")

In [ ]:
import matplotlib.pyplot as plt
import json
from joblib import Parallel, delayed, parallel_backend
from multiprocessing import Process, Lock
from tqdm import tqdm
def read_dic(path):
    a_file = open(path, "r")
    output = a_file.read()
    return output

def write(data, path):
    a_file = open(path, "w")
    json.dump(data, a_file)
    a_file.close()

submissions = {}
    
def process(id, lock):
    location = "../Project_CodeNet/metadata/"+id+'.csv'
    problem_df = pd.read_csv(location)
    #print(problem_df.columns)
    for inddex, row in tqdm(problem_df.iterrows()):
        information_tuple = (row['submission_id'], row['date'],row['language'],row['original_language'], row['filename_ext'], row['status'])
        lock.acquire()
        if(row['user_id'] in submissions.keys()):
            if(row['problem_id'] in submissions[row['user_id']].keys()):
                submissions[row['user_id']][row['problem_id']].append(information_tuple)
            else:
                submissions[row['user_id']][row['problem_id']] = [information_tuple]
        else:
            submissions[row['user_id']] = {}
            submissions[row['user_id']][row['problem_id']] = [information_tuple]
        
        lock.release()
    #from pprint import pprint
    #pprint(submissions)
'''
['submission_id', 'problem_id', 'user_id', 'date', 'language',
       'original_language', 'filename_ext', 'status', 'cpu_time', 'memory',
       'code_size', 'accuracy'],
'''  

lock = Lock()

Parallel(n_jobs=36, prefer="threads")(
    delayed(process)(row['id'], lock) for index, row in problemListCsvDF.iterrows())
'''
for index, row in problemListCsvDF.iterrows():
    print(index, row['id'])
    process(row['id'])
    break
'''
write(submissions, "processed.json")

In [ ]:
cnt = 0
for user in submissions:
    for problem in submissions[user]:
        cnt+=len(submissions[user][problem])

Process json

In [ ]:
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import json
from joblib import Parallel, delayed, parallel_backend
from multiprocessing import Process, Lock
from tqdm import tqdm

import json
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
data = getJsonData("processed.json")
ans = 0
for user in tqdm(data.keys()):
    for problem_id in data[user].keys():
        ans+=len(data[user][problem_id])
print("total data = ", ans)

In [ ]:
from difflib import SequenceMatcher
from codegen.preprocessing.lang_processors.java_processor import JavaProcessor
from codegen.preprocessing.lang_processors.python_processor import PythonProcessor

root_folder = "../third_party"
jprocessor = JavaProcessor(root_folder=root_folder)
pyprocessor = PythonProcessor(root_folder=root_folder)

class Solution:
    def __init__(
            self,
            source,
            lang,
            problem_id,
            code_tokens,
            functions_standalone=[],
            functions_class=[],
            submission_id="",
            verdict=""
    ):
        self.source = source
        self.lang = lang
        self.problem_id = problem_id
        self.code_tokens = code_tokens
        self.submission_id = submission_id
        self.functions_standalone = functions_standalone
        self.functions_class = functions_class
        self.verdict=verdict

    def __repr__(self):
        return 'source: ' + self.source + '\n' + \
               'lang: ' + self.lang + '\n' + \
               'problem_id: ' + self.problem_id + '\n' + \
               'code: ' + self.code_tokens + '\n' + \
               'submission_id: ' + self.submission_id + '\n' + \
               'verdict: ' + self.verdict + '\n'

    def __str__(self):
        return 'source: ' + self.source + '\n' + \
               'lang: ' + self.lang + '\n' + \
               'problem_id: ' + self.problem_id + '\n' + \
               'code: ' + self.code_tokens + '\n' + \
               'submission_id: ' + self.submission_id + '\n'+ \
               'verdict: ' + self.verdict + '\n'
    def toJSON(self):
        return self.__dict__


def calculate_similarity(code1_tokens, code2_tokens):
    code1 = ' '.join(code1_tokens)
    code2 = ' '.join(code2_tokens)
    return SequenceMatcher(None, code1, code2).ratio()

def write_output_to_file(problems, file_to_write):
    mode = 'w'
    if os.path.exists(file_to_write):
        mode = 'a'
        with open(file_to_write, mode, encoding='utf8') as fw:
            fw.write('\n')
    with open(file_to_write, mode, encoding='utf8') as fw:
        fw.write('\n'.join([json.dumps(p) for p in problems]))



def process_user(user_id):
    java_solutions = []
    python_solutions = []

    for problem_id in data[user_id].keys():
        submissions = data[user_id][problem_id]

        acceptedJavaSolution = ""
        closeJavaSolution = ""
        acceptedPythonSolution = ""
        closePythonSolution = ""

        functions_standalone_py=[]
        functions_class_py = []
        functions_standalone_java=[]
        functions_class_java = []

        closeSubmissionIdpy = ""
        closeSubmissionIdjava=""
        submission_id_py=""
        submission_id_java=""
        verdict_py=""
        verdict_java=""

        for submisson in submissions:
            try:
                if(submisson[2]=="Java"):
                    if(submisson[5]=="Accepted"):
                        solution_path = "../Project_CodeNet/data/"+problem_id+"/Java/"+submisson[0]+'.java'
                        with open(solution_path, 'r', encoding='utf8') as f:
                            code = f.read()
                            acceptedJavaSolution = code[::]
                            submission_id_java = submisson[0]
                            code_tokens_java = jprocessor.tokenize_code(code)
                            fn_standalone_java, fn_class_java = jprocessor.extract_functions(code_tokens_java)
                            functions_standalone_java = [(jprocessor.get_function_name(fn), fn) for fn in fn_standalone_java]
                            functions_class_java = [(jprocessor.get_function_name(fn), fn) for fn in fn_class_java]


                if(submisson[2]=="Python"):
                    if(submisson[5]=="Accepted"):
                        solution_path = "../Project_CodeNet/data/"+problem_id+"/Python/"+submisson[0]+'.py'
                        with open(solution_path, 'r', encoding='utf8') as f:
                            code = f.read()
                            acceptedPythonSolution = code[::]
                            submission_id_py = submisson[0]
                            code_tokens_py = pyprocessor.tokenize_code(code)
                            fn_standalone_py, fn_class_py = pyprocessor.extract_functions(code_tokens_py)
                            functions_standalone_py = [(pyprocessor.get_function_name(fn), fn) for fn in fn_standalone_py]
                            functions_class_py = [(pyprocessor.get_function_name(fn), fn) for fn in fn_class_py]
            except BaseException as error:
                print("error for submission top problem id",problem_id, submisson)
                pass


        java_min_distance = 999999
        python_min_distance = 999999
        
        for submisson in submissions:
            try:
                if(submisson[2]=="Java"):
                    if(submisson[5]!="Accepted"):
                        solution_path = "../Project_CodeNet/data/"+problem_id+"/Java/"+submisson[0]+'.java'
                        with open(solution_path, 'r', encoding='utf8') as f:
                            code = f.read()
                            dis = calculate_similarity(code,acceptedJavaSolution)
                            if(dis<java_min_distance):
                                closeJavaSolution=code[::]
                                java_min_distance = dis
                                closeSubmissionIdjava = submisson[0]
                                verdict_java=submisson[5]


                if(submisson[2]=="Python"):
                    if(submisson[5]!="Accepted"):
                        solution_path = "../Project_CodeNet/data/"+problem_id+"/Python/"+submisson[0]+'.py'
                        with open(solution_path, 'r', encoding='utf8') as f:
                            code = f.read()
                            dis = calculate_similarity(code,acceptedPythonSolution)
                            if(dis<python_min_distance):
                                closePythonSolution=code[::]
                                python_min_distance = dis
                                closeSubmissionIdpy = submisson[0]
                                verdict_py=submisson[5]
            except BaseException as error:
                print("error for submission ", submisson)
                pass
            
        if(len(acceptedPythonSolution)*len(closePythonSolution)):  
            python_solutions.append((Solution(
                                        "source",
                                        'python',
                                        problem_id,
                                        closePythonSolution,
                                        functions_standalone_py,
                                        functions_class_py,
                                        closeSubmissionIdpy,
                                        verdict_py
                                    ).toJSON(),Solution(
                                        "source",
                                        'python',
                                        problem_id,
                                        acceptedPythonSolution,
                                        functions_standalone_py,
                                        functions_class_py,
                                        submission_id_py,
                                        "Accepted"
                                    ).toJSON())
                                )
        if(len(acceptedJavaSolution)*len(closeJavaSolution)):
            java_solutions.append((Solution(
                                        "source",
                                        'java',
                                        problem_id,
                                        closeJavaSolution,
                                        functions_standalone_java,
                                        functions_class_java,
                                        closeSubmissionIdjava,
                                        verdict_java
                                    ).toJSON(),Solution(
                                        "source",
                                        'java',
                                        problem_id,
                                        acceptedJavaSolution,
                                        functions_standalone_java,
                                        functions_class_java,
                                        submission_id_java,
                                        "Accepted"
                                    ).toJSON())
                                )
        #write in a file next task
    write_output_to_file(java_solutions, "java_solutions.jsonl")
    write_output_to_file(python_solutions, "python_solutions.jsonl")
    

In [ ]:
for user in tqdm(list(data.keys())[:10]):
    process_user(user)

In [ ]:
print(len(java_solutions))
print(len(python_solutions))

In [ ]:
python_solutions[-1]

In [ ]:
#cat ../Project_CodeNet/data/p00220/Java/s289306249.java
#cat ../Project_CodeNet/data/p00442/Java/s813585875.java
from codegen.preprocessing.lang_processors.java_processor import JavaProcessor
from codegen.preprocessing.lang_processors.python_processor import PythonProcessor

root_folder = "../third_party"
jprocessor = JavaProcessor(root_folder=root_folder)
#sol = "../Project_CodeNet/data/p00220/Java/s289306249.java"
#sol = "exceptional_case.java"
sol = "../Project_CodeNet/data/p04049/Java/s881422316.java"

with open(sol, 'r', encoding='utf8') as f:
    code = f.read()
    print(code.strip())
    code_tokens_java = jprocessor.tokenize_code(code)
    fn_standalone_java, fn_class_java = jprocessor.extract_functions(code_tokens_java)
    print(fn_standalone_java, fn_class_java)
    functions_standalone_java = [(jprocessor.get_function_name(fn), fn) for fn in fn_standalone_java]
    functions_class_java = [(jprocessor.get_function_name(fn), fn) for fn in fn_class_java]

In [ ]:
with open('exceptional_case.java', 'r', encoding='utf8') as f:
    code = f.read()
    print(code)

In [ ]:
with open("python_solutions.jsonl", 'r') as f:
    for line in f:
        ex = json.loads(line)
        pprint(ex[1])
        break

In [ ]:
cp ../Project_CodeNet/data/p00220/Java/s289306249.java exceptional_case.java

In [ ]:
ls

In [ ]:
import json
from pprint import pprint
with open("exceptional.jsonl", 'r') as f:
    for line in f:
        #print(line.replace("\'","\""))
        ex = json.loads(line.replace("\'","\""))
        pprint(ex)
        break

In [ ]:
def write(data, path):
    a_file = open(path, "w")
    json.dump(data, a_file)
    a_file.close()
dt = {
    'key':'val',
    'key1':'va"aba"bl',
}
write([dt, dt], "excep.json")

In [ ]:

import json
from pprint import pprint
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
java_data = getJsonData('../data/java/jsons/0.json')
#python_data = getJsonData('../data/Python/jsons/0.json')

In [ ]:
pprint(java_data[1])

In [ ]:
for val in java_data:
    print(val[0]['code_tokens'])
    print("################")
    print(val[1]['code_tokens'])
    break

In [ ]:
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import json
from joblib import Parallel, delayed, parallel_backend
from multiprocessing import Process, Lock
from tqdm import tqdm
import os
import json
import threading

def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data

data = getJsonData("processed.json")
ans = 0
for user in tqdm(data.keys()):
    for problem_id in data[user].keys():
        ans+=len(data[user][problem_id])
print("total data = ", ans)


In [ ]:

from difflib import SequenceMatcher
from codegen.preprocessing.lang_processors.java_processor import JavaProcessor
from codegen.preprocessing.lang_processors.python_processor import PythonProcessor
import timeout_decorator 
 

root_folder = "../third_party"
jprocessor = JavaProcessor(root_folder=root_folder)
pyprocessor = PythonProcessor(root_folder=root_folder)

class Solution:
    def __init__(
            self,
            source,
            lang,
            problem_id,
            code_tokens,
            functions_standalone=[],
            functions_class=[],
            submission_id="",
            verdict=""
    ):
        self.source = source
        self.lang = lang
        self.problem_id = problem_id
        self.code_tokens = code_tokens.replace("\"", "")
        self.submission_id = submission_id
        self.functions_standalone = functions_standalone
        self.functions_class = functions_class
        self.verdict=verdict

    def __repr__(self):
        return 'source: ' + self.source + '\n' + \
               'lang: ' + self.lang + '\n' + \
               'problem_id: ' + self.problem_id + '\n' + \
               'code: ' + self.code_tokens + '\n' + \
               'submission_id: ' + self.submission_id + '\n' + \
               'verdict: ' + self.verdict + '\n'

    def __str__(self):
        return 'source: ' + self.source + '\n' + \
               'lang: ' + self.lang + '\n' + \
               'problem_id: ' + self.problem_id + '\n' + \
               'code: ' + self.code_tokens + '\n' + \
               'submission_id: ' + self.submission_id + '\n'+ \
               'verdict: ' + self.verdict + '\n'
    def toJSON(self):
        return self.__dict__

@timeout_decorator.timeout(0.2) 
def calculate_similarity(code1_tokens, code2_tokens):
    code1 = ' '.join(code1_tokens)
    code2 = ' '.join(code2_tokens)
    return SequenceMatcher(None, code1, code2).ratio()

#Lock = threading.Lock()

def write_output_to_file(problems, file_to_write):
    Lock.acquire()
    mode = 'w'
    if os.path.exists(file_to_write):
        mode = 'a'
        with open(file_to_write, mode, encoding='utf8') as fw:
            fw.write('\n')
    with open(file_to_write, mode, encoding='utf8') as fw:
        fw.write('\n'.join([json.dumps(p) for p in problems]))
    Lock.release()

java_file_id = 0
python_file_id = 0
java_solutions = []
python_solutions = []

def write_output_to_json_file(file_name, one_example):
    global java_solutions
    global python_solutions
    global java_file_id
    global python_file_id
    #global Lock
    if(file_name =='Java'):
        #Lock.acquire()
        java_solutions.append(one_example)
        #print("java len", len(java_solutions))
        if(len(java_solutions)==10000):
            pth = '../data/Java/jsons/{}.json'.format(java_file_id)
            with open(pth, 'w+', encoding="utf8") as f:
                json.dump(java_solutions, f)
            java_solutions=[]
            java_file_id+=1
        #Lock.release()
    if(file_name=='Python'):
        #Lock.acquire()
        #print("python len", len(python_solutions))
        python_solutions.append(one_example)
        if(len(python_solutions)==10000):
            pth = '../data/Python/jsons/{}.json'.format(python_file_id)
            with open(pth, 'w+', encoding="utf8") as f:
                json.dump(python_solutions, f)
            python_solutions=[]
            python_file_id+=1
        #Lock.release()
    



def process_user(user_id):

    for problem_id in data[user_id].keys():
        print("problem id = ", problem_id)
        submissions = data[user_id][problem_id]

        acceptedJavaSolution = ""
        closeJavaSolution = ""
        acceptedPythonSolution = ""
        closePythonSolution = ""

        functions_standalone_py=[]
        functions_class_py = []
        functions_standalone_java=[]
        functions_class_java = []

        closeSubmissionIdpy = ""
        closeSubmissionIdjava=""
        submission_id_py=""
        submission_id_java=""
        verdict_py=""
        verdict_java=""

        for submisson in submissions:
            try:
                #print("in submission 152")
                if(submisson[2]=="Java"):
                    if(submisson[5]=="Accepted"):
                        solution_path = "../Project_CodeNet/data/"+problem_id+"/Java/"+submisson[0]+'.java'
                        with open(solution_path, 'r', encoding='utf8') as f:
                            code = f.read()
                            acceptedJavaSolution = code[::]
                            submission_id_java = submisson[0]
                            #code_tokens_java = jprocessor.tokenize_code(code)
                            #fn_standalone_java, fn_class_java = jprocessor.extract_functions(code_tokens_java)
                            #functions_standalone_java = [(jprocessor.get_function_name(fn), fn) for fn in fn_standalone_java]
                            #functions_class_java = [(jprocessor.get_function_name(fn), fn) for fn in fn_class_java]


                if(submisson[2]=="Python"):
                    if(submisson[5]=="Accepted"):
                        solution_path = "../Project_CodeNet/data/"+problem_id+"/Python/"+submisson[0]+'.py'
                        with open(solution_path, 'r', encoding='utf8') as f:
                            code = f.read()
                            acceptedPythonSolution = code[::]
                            submission_id_py = submisson[0]
                            #code_tokens_py = pyprocessor.tokenize_code(code)
                            #fn_standalone_py, fn_class_py = pyprocessor.extract_functions(code_tokens_py)
                            #functions_standalone_py = [(pyprocessor.get_function_name(fn), fn) for fn in fn_standalone_py]
                            #functions_class_py = [(pyprocessor.get_function_name(fn), fn) for fn in fn_class_py]
            except BaseException as error:
                print("error for submission top problem id",problem_id, submisson)
                pass
        
        java_min_distance = 999999
        python_min_distance = 999999
        
        for submisson in submissions:
            #print("in submission 185 submission id:", submisson)
            try:
                if(submisson[2]=="Java"):
                    if(submisson[5]!="Accepted"):
                        solution_path = "../Project_CodeNet/data/"+problem_id+"/Java/"+submisson[0]+'.java'
                        with open(solution_path, 'r', encoding='utf8') as f:
                            code = f.read()
                            dis = calculate_similarity(code,acceptedJavaSolution)
                            if(dis<java_min_distance):
                                closeJavaSolution=code[::]
                                java_min_distance = dis
                                closeSubmissionIdjava = submisson[0]
                                verdict_java=submisson[5]


                if(submisson[2]=="Python"):
                    if(submisson[5]!="Accepted"):
                        solution_path = "../Project_CodeNet/data/"+problem_id+"/Python/"+submisson[0]+'.py'
                        with open(solution_path, 'r', encoding='utf8') as f:
                            code = f.read()
                            dis = calculate_similarity(code,acceptedPythonSolution)
                            if(dis<python_min_distance):
                                closePythonSolution=code[::]
                                python_min_distance = dis
                                closeSubmissionIdpy = submisson[0]
                                verdict_py=submisson[5]
            except BaseException as error:
                print("error for submission ", submisson)
                pass

        if(len(acceptedPythonSolution)*len(closePythonSolution)):  
            write_output_to_json_file("Python",(Solution(
                                        "source",
                                        'python',
                                        problem_id,
                                        closePythonSolution,
                                        functions_standalone_py,
                                        functions_class_py,
                                        closeSubmissionIdpy,
                                        verdict_py
                                    ).toJSON(),Solution(
                                        "source",
                                        'python',
                                        problem_id,
                                        acceptedPythonSolution,
                                        functions_standalone_py,
                                        functions_class_py,
                                        submission_id_py,
                                        "Accepted"
                                    ).toJSON())
                                )
        if(len(acceptedJavaSolution)*len(closeJavaSolution)):
            write_output_to_json_file("Java", (Solution(
                                        "source",
                                        'java',
                                        problem_id,
                                        closeJavaSolution,
                                        functions_standalone_java,
                                        functions_class_java,
                                        closeSubmissionIdjava,
                                        verdict_java
                                    ).toJSON(),Solution(
                                        "source",
                                        'java',
                                        problem_id,
                                        acceptedJavaSolution,
                                        functions_standalone_java,
                                        functions_class_java,
                                        submission_id_java,
                                        "Accepted"
                                    ).toJSON())
                                )
        #write in a file next task
    '''
    if(len(java_solutions)):
        write_output_to_file(java_solutions, "java_solutions.jsonl")
    if(len(python_solutions)):
        write_output_to_file(python_solutions, "python_solutions.jsonl")
    '''


In [ ]:
process_user('u326609687')

In [ ]:
data['u326609687']['p02574']

In [ ]:
data['u326609687']['p02574']

In [ ]:
#cat ../Project_CodeNet/data/p02574/Python/s422117152.py
!cat ../Project_CodeNet/data/p02575/Python/s428831120.py



In [ ]:
ls ../codet5/tensorboard/home/mahim/program_repair/CodeNet/codet5/codenet/java2java/

In [ ]:
codet5/tensorboard/home/mahim/program_repair/CodeNet/codet5/codenet/java2j

In [ ]:
import json
with open("/home/mahim/program_repair/CodeNet/data/java/processed/test.jsonl", encoding="utf8") as f:
    data = json.load(f)
    #print(len(data))
    print(data[0].keys())


Some statistics

In [ ]:
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import json
from joblib import Parallel, delayed, parallel_backend
from multiprocessing import Process, Lock
from tqdm import tqdm

import json
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
data = getJsonData("processed.json")
ans = 0
for user in tqdm(data.keys()):
    for problem_id in data[user].keys():
        ans+=len(data[user][problem_id])
print("total data = ", ans)

In [ ]:
import matplotlib.pyplot as plt
Problem_count = 0

lang_count = 0
path_sum = 0
solved_be_user = 0

path_lengths = []
submission_counts = []
lang_counts = []
for user in tqdm(list(data.keys())):
    language_lst=set()
    submission_count = 0
    for problem_id in data[user].keys():
        path_len = 0
        if(len(data[user][problem_id])):
            Problem_count+=1
        
        langs = {}
        for idx, subs in enumerate(data[user][problem_id]):
            # need to consider for each language
            if subs[2] in langs.keys():
                langs[subs[2]].append(subs)
            else:
                langs[subs[2]] = [subs]
            language_lst.add(subs[2])
            submission_count+=1
        
        # Path sum calculation
        for lang in langs:
            first_wa = 0
            first_ac = -1
            for idx, sub in enumerate(sorted(langs[lang], key=lambda x: x[1])):
                if(sub[-1]=="Accepted"):
                    first_ac = max(first_ac, idx)
                else:
                    first_wa = min(first_wa, idx)
        
            if(first_ac-first_wa+1>0):
                #print(langs[lang])
                #print(first_ac-first_wa)    
                path_sum+=first_ac-first_wa+1
                solved_be_user+=1
                path_lengths.append(first_ac-first_wa+1)

            
    lang_count+=len(language_lst)
    lang_counts.append(len(language_lst))
    submission_counts.append(submission_count)
        
    #break
print("per user average submission count", sum(submission_counts)/len(data))
print("per user average language usage count", lang_count/len(data))
print("per user average accepted path length", path_sum/solved_be_user)
'''
per user average submission count 90.26494052329126
per user average language usage count 1.4337000090804135
per user average accepted path length 1.9145810148516795
'''

In [ ]:
n_bins=10
n, bins, patches = plt.hist(path_lengths, bins=[i for i in range(11)])
ticks = [(patch._x0 + patch._x1)/2 for patch in patches]
ticklabels = [i for i in range(n_bins)]
plt.xticks(ticks, ticklabels)
plt.xlabel("Number of submissions before accepted")
plt.ylabel("Number of users")
plt.show()

In [ ]:
n_bins=10
n, bins, patches = plt.hist(lang_counts, bins=[i for i in range(11)])
ticks = [(patch._x0 + patch._x1)/2 for patch in patches]
ticklabels = [i for i in range(n_bins)]
plt.xticks(ticks, ticklabels)
plt.xlabel("Number of Languages used by a user")
plt.ylabel("Number of users")
plt.show()

In [ ]:

n, bins, patches = plt.hist(submission_counts, bins=[i for i in range(200)])
#ticks = [(patch._x0 + patch._x1)/2 for patch in patches]
#ticklabels = [i for i in range(n_bins)]
#plt.xticks(ticks, ticklabels)
plt.xlabel("Number of Submissons per user")
plt.ylabel("Number of users")
plt.show()

In [ ]:
submission_counts[:100]

In [ ]:
import json
from glob import glob

files = "../data/java/jsons/"
src_sizes = []
tgt_sizes = []
for file in glob(files+'*.json'):
    print(file)
    with open(file, 'r') as f:
        temp = json.load(f)
        for ex in temp:
            src_sizes.append(len(ex[0]['code_tokens']))
            tgt_sizes.append(len(ex[1]['code_tokens']))

In [ ]:
import matplotlib.pyplot as plt
plt.hist(src_sizes, bins=[i for i in range(0, 10000, 1000)])
plt.show()

In [ ]:
root = "../../BIFI/"

In [ ]:
import json
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
data = getJsonData("../../BIFI/data/orig_bad_code/orig.bad.json")

In [ ]:
from pprint import pprint
for key in list(data.keys())[:2]:
    pprint(data[key])

In [ ]:
import json
from glob import glob
from pprint import pprint
file = "../data/java/processed/train.jsonl"

with open(file, 'r') as f:
    train = json.load(f)
print(len(train))
    

In [ ]:
from pprint import pprint
from tqdm import tqdm
from difflib import SequenceMatcher
def calculate_similarity(code1_tokens, code2_tokens):
    code1 = ' '.join(code1_tokens)
    code2 = ' '.join(code2_tokens)
    return SequenceMatcher(None, code1, code2).ratio()
dis = []
for dt in tqdm(train):
    dis.append(calculate_similarity(dt['src'], dt['tgt']))


In [ ]:
import matplotlib.pyplot as plt
plt.hist(dis, bins=5)
plt.show()

In [ ]:
from glob import glob
problem_desc = []
for file in glob("../Project_CodeNet/problem_descriptions/*html"):
    problem_desc.append(file)

In [ ]:
problem_desc[0]

In [ ]:
try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup
Base_link = "https://www.dropbox.com/sh/nx3tnilzqz7df8a/AAAYlTq2tiEHl5hsESw6-yfLa?dl=0"
import requests
req = requests.get(Base_link)
#print(html)
soup = BeautifulSoup(req.text, "html.parser")
for a in soup.find_all('a', href=True):
    print("Found the URL:", a['href'])

In [ ]:
Access_token = "sl.BBQsZOVExLY-32FcgPQGwk5yMVeJCjShnddXfJGjzAnj0CQ-5GlVrRO39hPX7lP6BnpyFw1_3pE7wPwWsorCjTSwTRRiBqMkGB2q1KMmjPgrD78wu2lHpbrPA_xRQlWZ42oxz2o"

In [ ]:
import re
pattern = "https://www.dropbox.com/sh/nx3tnilzqz7df8a/.*/.*dl=0"

re.findall(r'{}'.format(pattern),req.text)

In [ ]:
import subprocess

def download_and_extract(lnk):
    lnk = lnk.replace("?dl=0", "?dl=1")
    name = lnk.split("/")[-1]
    name = name.split("?dl=1")[0]+".zip"
    #print(name)
    subprocess.run(["wget", "-O", name, lnk])
    subprocess.run(["unzip",name ,"-d","atcoder_test_cases/"+name.replace(".zip", "")])
    subprocess.run(["rm" ,"-rf",name])
    
    

In [ ]:
from tqdm import tqdm
with open("atcoder_links.txt") as f:
    links = f.readlines()
for link in tqdm(links):
    download_and_extract(link)

In [ ]:
import pandas as pd
problemlist=pd.read_csv("../Project_CodeNet/metadata/problem_list.csv")


In [ ]:
from collections import defaultdict
from pprint import pprint
from tqdm import tqdm
problems = defaultdict(list)
for index, row in tqdm(problemlist.iterrows()):
    if(row['dataset']=='AtCoder'):
        if("AtCoder Regular Contest" in row['name']):
            number = row['name'].split(" ")[3]
            problems["ARC"+number].append(row['id'])
        if("AtCoder Beginner Contest" in row['name']):
            number = row['name'].split(" ")[3]
            problems["ABC"+number].append(row['id'])
        if("AtCoder Grand Contest" in row['name']):
            number = row['name'].split(" ")[3]
            problems["AGC"+number].append(row['id'])

In [ ]:
from glob import glob
folders = glob("atcoder_test_cases/*")

final_keys = []
for idx in range(len(folders)):
    folders[idx] = folders[idx].replace("atcoder_test_cases/", "")
#print(folders)
for key in problems.keys():
    if key in folders:
        if len(problems[key]) == len(glob("atcoder_test_cases/"+key+"/*")):
            final_keys.append(key)
            
    elif key.lower() in folders :
        if len(problems[key]) == len(glob("atcoder_test_cases/"+ key.lower() +"/*")):
            final_keys.append(key)


In [ ]:
problemid_to_tc = {}
for key in problems:
    if(key in final_keys):
        for idx, prob_id in enumerate(problems[key]):
            folder_list = glob("atcoder_test_cases/"+key+"/*")
            if(len(folder_list)==0):
                folder_list = glob("atcoder_test_cases/"+key.lower()+"/*")
            problemid_to_tc[prob_id] = folder_list[idx]

In [ ]:
len(problemid_to_tc)

In [ ]:
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import json
from joblib import Parallel, delayed, parallel_backend
from multiprocessing import Process, Lock
from tqdm import tqdm

import json
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
data = getJsonData("../data/java/processed/test.jsonl")


In [ ]:
#data[0]
#def run_and_report()

#st = "atcoder_test_cases/ABC118/B/in/3_mmin_1"
#out = st.split("/")
#out[3] = "out"
#"/".join(out)

In [ ]:
from codegen.preprocessing.lang_processors.java_processor import JavaProcessor
from codegen.preprocessing.lang_processors.python_processor import PythonProcessor
import subprocess
import filecmp
from subprocess import Popen, PIPE, STDOUT
import sys

def check_floating(n1, n2):
    n1 = n1.replace("-", "").replace("+", "")
    n2 = n2.replace("-", "").replace("+", "")
    
    if (not n1.replace('.','',1).replace("E","").isdigit()) or (not n1.replace('.','',1).replace("E","").isdigit()):
        return False
    if abs(float(n1)-float(n2))<1e8:
        return True
    return False
    
def print_error(l1, l2):
    print("###")
    print(l1)
    print("###")
    print(l2)
    print("###")

def compare_files(file1, file2):
    with open(file1) as f1, open(file2) as f2: 
        content1 = f1.read().split()
        content2 = f2.read().split()

        for l1, l2 in zip(content1, content2):
            if l1.strip() != l2.strip(): 
                num1s = l1.strip().split(" ")
                num2s = l2.strip().split(" ")
                if(len(num1s) == len(num2s)):
                    for idx in range(len(num1s)):
                        if not check_floating(num1s[idx],num2s[idx]):
                            print_error(l1, l2)
                            return False
                else:
                    print_error(l1, l2)
                    return False
    return True

root_folder = "../third_party"
jprocessor = JavaProcessor(root_folder=root_folder)

uniq = set()

#code_tokens_java = jprocessor.tokenize_code(code)
cnt = 0
done = False
for dt in tqdm(data[12475:]): #12480+721 14975
    if dt['id'].split("_")[0] in problemid_to_tc.keys():
        #if(dt['tgt_id'].split("_")[1] != "s473235135"):
        #    continue
        cnt+=1
        #print("comes")
        #print(dt['id'])
        #pprint(dt)
        #print(dt['src'])
        code = jprocessor.detokenize_code(dt['tgt'])
        if dt['tgt_id'] in uniq:
            continue
        uniq.add(dt['tgt_id'])
        with open('Main.java', 'w', encoding='utf8') as fw:
            fw.write(code)
        test_case_folder = problemid_to_tc[dt['id'].split("_")[0]]
        in_files = glob(test_case_folder+"/in/*")
        p1 = subprocess.run(["javac","Main.java"])
        return_code = p1.returncode

        if(return_code):
            print("######## doesnt compile  ############")
            print(p1.stderr.decode("utf-8")) 14975
            print(dt['tgt_id'])
            #pprint(dt)
            #doesn't compile correctly

        for in_file in in_files:
            #subprocess.run(["java","Main" ,"<",in_files[0], ">", "cmd_out.txt"], shell=True)
            cmd = "java Main < {} > cmd_out.txt".format(in_file)
            p = Popen(cmd, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True)
            p.wait()
            #output = p.stdout
            #print(output)
            out = in_file.split("/")
            out[3] = "out"
            out_file ="/".join(out)
            out_file = out_file.replace(".in", ".out")

            p2 = subprocess.Popen(["cp",out_file, "cmd_out_match.txt"])
            p2.wait()
            #match 
            #print(filecmp.cmp('cmd_out.txt', 'cmd_out_match.txt'))
            if not compare_files('cmd_out.txt', 'cmd_out_match.txt'):
                #done=True
                print(dt['tgt_id'])
                #break
                print(in_file)
        p3 = subprocess.run(["rm","Main.java"])
        p4 = subprocess.run(["rm","*.class"])
#print(cnt)

#accept one valid way we need to skip those p02697,
#used weird library runs on jdk 1.8 but not in jdk 11 p02682_s116318276

In [ ]:
s = "p03334,p02678,p02690,p02692,p02697,p02850,p02825,p02902,p02903,p02976,p02977,p03097,p03938,p03245,p03044,p02972,p03263,p03836,p03267,p03257,p03394,p03952,p04007,p04035,p03746,p02963,p03095,p02669,p02825,p03046,p02997,p02936, p02694, p02975,p02731, p02861,p03110"
print(list(set(s.replace(" ", "").split(","))))

In [ ]:
p1 = subprocess.Popen(["javac","Main.java"])
return_code = p1.wait()
print(return_code)

In [ ]:
cmd = "java Main < atcoder_test_cases/ABC138/D/in/b04 > cmd_out.txt"
p = Popen(cmd, shell=True, stdin=PIPE, stdout=PIPE, stderr=subprocess.DEVNULL, close_fds=True)
p.wait()
print("command exucuted")

in_file = "atcoder_test_cases/ABC138/D/in/b04"
out = in_file.split("/")
out[3] = "out"
out_file ="/".join(out)
out_file = out_file.replace(".in", ".out")

p2 = subprocess.Popen(["cp",out_file, "cmd_out_match.txt"])
p2.wait()
print("Matching Files")

if not compare_files('cmd_out.txt', 'cmd_out_match.txt'):
    #done=True
    #print(dt['tgt_id'])
    print(in_file)

In [ ]:
#One such Way p02678,p02690,p02692,p02697,p02850,p02825,p02902,p02903,p02976,p02977,p03097,p03938,p03245,p03044,p02972
#One such way p03263,p03836,p03267,p03257,p03394,p03952,p04007,p04035,p03746,p02963,p03095,p02669,p02825,p03046,p02997

# p02936, p02694 p02975 one after contest doesn't match
#p02731 has higher thresholds for checking double value, p02861,p03110 as well

'''
Need to check 

p02912_s201342530
atcoder_test_cases/ABC141/D/in/testcase_03.nkftmpjKHWPL
this has one weird input which i deleted from the atcoder_testcases folder

'''

###
#num1 = "0.0000000000000001E-5"
#num2 = "-0.0000000000000001E-5"

#num1 = "1.1551382117326875E-5"
#num2 = "0.00001155138223854135\n"

###
#756.26049783473536042289
num1 = "3.57508992E8"
###
num2 = "357508993.7616"
def only_digits(num):
    return num.replace("-", "").replace("+", "").replace('.','',1).replace("E","").isdigit()

def check_floating(n1, n2):
    if (not only_digits(n1)) or (not only_digits(n2)):
        return False
    print(float(n1), float(n2))
    if abs(float(n1)-float(n2))<1e-6:
        return True
    return False
    
num1s = num1.strip().split(" ")
num2s = num2.strip().split(" ")
if(len(num1s) != len(num2s)):
    print("Not equal")
for idx in range(len(num1s)):
    print(check_floating(num1s[idx],num2s[idx]))


In [ ]:
float("0.00001155138223854135")

In [ ]:
def check_floating(n1, n2):
    n1 = n1.replace("-", "").replace("+", "")
    n2 = n2.replace("-", "").replace("+", "")
    
    if (not n1.replace('.','',1).replace("E","").isdigit()) or (not n1.replace('.','',1).replace("E","").isdigit()):
        return False
    if abs(float(n1)-float(n2))<1e8:
        return True
    return False
    
def print_error(l1, l2):
    print("###")
    print(l1)
    print("###")
    print(l2)
    print("###")

def compare_files(file1, file2):
    try:
        with open(file1) as f1, open(file2) as f2: 
            content1 = f1.read().split()
            content2 = f2.read().split()
            for l1, l2 in zip(content1, content2):
                if l1.strip() != l2.strip(): 
                    num1s = l1.strip().split(" ")
                    num2s = l2.strip().split(" ")
                    if(len(num1s) == len(num2s)):
                        for idx in range(len(num1s)):
                            if not check_floating(num1s[idx],num2s[idx]):
                                print_error(l1, l2)
                                return False
                    else:
                        print_error(l1, l2)
                        return False
            return True
    except Exception as e:
        print("exception = ", e)
        return False


In [ ]:
!head atcoder_test_cases/ABC157/A/out/01-handmade-01
p02658_s473235135

In [ ]:
#!rm temp.java
p03037_s170937198
!cp ../Project_CodeNet/data/p02658/Java/s473235135.java temp2.java

In [ ]:
def check_floating(n1, n2):
    n1 = n1.replace("-", "").replace("+", "")
    n2 = n2.replace("-", "").replace("+", "")
    
    if (not n1.replace('.','',1).replace("E","").isdigit()) or (not n1.replace('.','',1).replace("E","").isdigit()):
        return False
    if abs(float(n1)-float(n2))<1e-8:
        return True
    return False
    
def print_error(l1, l2):
    print("###")
    print(l1)
    print("###")
    print(l2)
    print("###")

def compare_files(file1, file2):
    with open(file1) as f1, open(file2) as f2: 
        content1 = f1.read().split()
        content2 = f2.read().split()
        #print(content1, content2)
        for l1, l2 in zip(content1, content2):
            print(l1.strip(), l2.strip(), l1.strip() != l2.strip())
            if l1.strip() != l2.strip(): 
                num1s = l1.strip().split(" ")
                num2s = l2.strip().split(" ")
                if(len(num1s) == len(num2s)):
                    for idx in range(len(num1s)):
                        if not check_floating(num1s[idx],num2s[idx]):
                            print_error(l1, l2)
                            return False
                else:
                    print_error(l1, l2)
                    return False
        return True
compare_files('cmd_out.txt', 'cmd_out_match.txt')

In [ ]:
p1 = subprocess.Popen("javac Main.java", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print("wait result", p1.wait())
output,error=p1.communicate()
print("###")
print(output)
print("###")
print(error)
print("####")
print(p1.returncode)


cmd = "java Main < {} > manual_check.txt".format("atcoder_test_cases/ABC169/B/in/max_02.txt")
p = Popen(cmd, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True)
p.wait()
print("ok")
            

In [ ]:
#problemid_to_tc['p02677']
ab = "a string '' c"
nn = ab[::]
print(nn)


In [ ]:
with open("temp2.java") as f:
    fl = f.read()
print(jprocessor.detokenize_code(jprocessor.tokenize_code(fl)))

In [ ]:
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import json
from joblib import Parallel, delayed, parallel_backend
from multiprocessing import Process, Lock
from tqdm import tqdm

import json
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
data = getJsonData("processed.json")
ans = 0
for user in tqdm(data.keys()):
    for problem_id in data[user].keys():
        #ans+=len(data[user][problem_id])
        for sub in data[user][problem_id]:
            if sub[0] == "s116318276":
                print(user)
                print(sub)
print("total data = ", ans)

In [ ]:
test = getJsonData("../data/java/processed/test.jsonl")
#for dt in test:
for user in tqdm(data.keys()):
    for problem_id in data[user].keys():
        for sub in data[user][problem_id]:
            #print(sub)
            if sub[0] == "s116318276":
                print(user,problem_id,sub )
                print(sub)
                break

In [ ]:
ze = getJsonData("../data/java/jsons/27.json")
for dt in ze:
    if(len(dt)<2):
        pprint(dt)

In [ ]:
for running java
javac test_java.java 
java test_java < cmd_in.txt > cmd_out.txt

for running python
python3 test_py.py < cmd_in.txt > cmd_out.txt

In [ ]:
import json
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
train = getJsonData("../data/java/processed/train.jsonl")
test = getJsonData("../data/java/processed/test.jsonl")
valid = getJsonData("../data/java/processed/valid.jsonl")

In [ ]:
from pprint import pprint
from tqdm import tqdm
problems = set()
for json in [train, test, valid]:
    for dt in tqdm(json):
        problems.add(dt['src_id'].split("_")[0])
#pprint(data[0]['src_id'])

In [ ]:
import json
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
processed = getJsonData("processed.json")

In [ ]:
from pprint import pprint
from tqdm import tqdm
java = set()
for key in tqdm(processed):
    pprint(processed[key])
    
    for problem in processed[key]:
        for sub in processed[key][problem]:
            if 'Python' in sub[2]:
                print(sub)
                break
                
    

In [ ]:
len(problems & java)